# Лабораторная работа №3

ФИО: **Павленко Алексей**
Группа: **Бивт-21-3**

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №7 в семестре (Занятие №3 очное)

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/11600OkqdRzxc-xZmFTuBs099VeAqU_GlE0Bx6bBlIRE/edit?usp=sharing) необходимо узнать название своего датасета
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1Phm-Fq1GL-VX7NS-DCEMRh_Fo1wU7DQ9?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`)
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

##**Пункт 1 ( Импорт библиотек. Пример импорта данных)**

In [ ]:
# data = pd.read_csv(path_to_dataset, sep='\t')  # если нужно, можно поменять разделитель
#  numpy уже импортирован в colab
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, f1_score, roc_curve, precision_recall_curve
from math import sqrt
import os
import pandas as pd
#from google.colab import files
#uploaded = files.upload()
from scipy import stats
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
data = pd.read_csv('Application_Data.csv', sep=',')
print(data.head())


   Applicant_ID Applicant_Gender  Owned_Car  Owned_Realty  Total_Children  \
0       5008806          M                1             1               0   
1       5008808          F                0             1               0   
2       5008809          F                0             1               0   
3       5008810          F                0             1               0   
4       5008811          F                0             1               0   

   Total_Income                                        Income_Type  \
0        112500  Working                                       ...   
1        270000  Commercial associate                          ...   
2        270000  Commercial associate                          ...   
3        270000  Commercial associate                          ...   
4        270000  Commercial associate                          ...   

                                      Education_Type  \
0  Secondary / secondary special                 ...   
1  S

##**Описание постановки задачи**

Цель:
Целью данной лабораторной работы является изучение и применение основных методов предобработки данных, обучения моделей машинного обучения и оценки их производительности.

Тип задачи:
Данная задача бинарной классификации включает предсказание результатов, которые имеют два возможных исхода. Примером служит определение того, будет ли кредитная карта одобрена или нет, основываясь на финансовой истории и других данных заявителей.

##**Пункт 3 (Разделение данных)**

In [ ]:
# Разделение данных на тренировочную и тестовую выборки
# Существует также верное разбиение на соотношение 70% к 30%

train = data[int(len(data)*0.8):]  # 80%
test = data[:int(len(data)*0.2)] # 20%
# Проверка пропусков в данных
print(train.isnull().sum())
train_missing_values = train.isnull().sum()
print(train_missing_values)

# Проверяем, есть ли хоть один пропуск в данных
if train_missing_values.sum() == 0:
    print("Нет пропусков данных")
else:
    print("Есть пропуски данных")

Applicant_ID            0
Applicant_Gender        0
Owned_Car               0
Owned_Realty            0
Total_Children          0
Total_Income            0
Income_Type             0
Education_Type          0
Family_Status           0
Housing_Type            0
Owned_Mobile_Phone      0
Owned_Work_Phone        0
Owned_Phone             0
Owned_Email             0
Job_Title               0
Total_Family_Members    0
Applicant_Age           0
Years_of_Working        0
Total_Bad_Debt          0
Total_Good_Debt         0
Status                  0
dtype: int64
Applicant_ID            0
Applicant_Gender        0
Owned_Car               0
Owned_Realty            0
Total_Children          0
Total_Income            0
Income_Type             0
Education_Type          0
Family_Status           0
Housing_Type            0
Owned_Mobile_Phone      0
Owned_Work_Phone        0
Owned_Phone             0
Owned_Email             0
Job_Title               0
Total_Family_Members    0
Applicant_Age           0

In [ ]:
# Определение численных и категориальных столбцов
num_cols = ['Owned_Car', 'Owned_Realty','Total_Children','Total_Income','Owned_Mobile_Phone','Owned_Work_Phone','Owned_Phone','Owned_Email','Total_Family_Members','Applicant_Age','Years_of_Working','Total_Bad_Debt','Total_Good_Debt','Status']
cat_cols = ['Applicant_Gender', 'Income_Type', 'Education_Type', 'Family_Status', 'Housing_Type','Job_Title']

# Создание преобразователя колонок
preprocessor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),('num', StandardScaler(), num_cols)]) #Student ignore

# Применение преобразований к тренировочным и тестовым данным
train_preprocessed = preprocessor.fit_transform(train)
test_preprocessed = preprocessor.transform(test)

print("Данные до преобразования:")
print(train.head())

# Преобразование в DataFrame (это не обязательно, но может быть полезно для дальнейшего анализа)
train_preprocessed_df = pd.DataFrame(train_preprocessed)
test_preprocessed_df = pd.DataFrame(test_preprocessed)

# Вывод преобразованных данных
print("\nРазделение по группам:")
print(preprocessor.get_feature_names_out(), "\n\n")
print("\nПреобразованные данные:")
print(train_preprocessed_df.head())


Данные до преобразования:
       Applicant_ID Applicant_Gender  Owned_Car  Owned_Realty  Total_Children  \
20102       5117824          M                1             0               0   
20103       5117832          M                1             0               0   
20104       5117834          M                1             0               0   
20105       5117835          M                1             0               0   
20106       5117836          M                1             0               0   

       Total_Income                                        Income_Type  \
20102        274500  Commercial associate                          ...   
20103        157500  Working                                       ...   
20104        157500  Working                                       ...   
20105        157500  Working                                       ...   
20106        157500  Working                                       ...   

                                          

##**Пункт 4 (Обучение на тренировочном множестве данных)**

In [ ]:
x = np.delete(train_preprocessed, len(num_cols) - 1, axis=1) # все столбцы, кроме ключевой фунции y
y = train_preprocessed[:, len(num_cols) - 1] # только ключевая функция y

# Линейные модели
linear_model = LogisticRegression()
linear_model.fit(x, y)
print("LinearRegression - Выполнено")

# Деревянные модели
tree_regressor = DecisionTreeClassifier(max_depth=2)
tree_regressor.fit(x, y)
print("DecisionTreeRegressor - Выполнено")

# K-ближайших соседей
knn_regressor = KNeighborsClassifier(n_neighbors=2)
knn_regressor.fit(x, y)
print("KNeighborsRegressor - Выполнено")

# Случайный лес
random_forest_regressor = RandomForestClassifier(n_estimators=10)
random_forest_regressor.fit(x, y)
print("RandomForestRegressor - Выполнено")

LinearRegression - Выполнено
DecisionTreeRegressor - Выполнено
KNeighborsRegressor - Выполнено
RandomForestRegressor - Выполнено


##**Пункт 5 (Подсчет метрик)**

In [ ]:
x_test = np.delete(test_preprocessed, len(num_cols) - 1, axis=1) # все столбцы, кроме test preparation course
y_test = test_preprocessed[:, len(num_cols) - 1] # только test preparation course

# Предсказания моделей
y_pred_linear = linear_model.predict(x)
y_pred_tree = tree_regressor.predict(x)
y_pred_knn = knn_regressor.predict(x)
y_pred_rf = random_forest_regressor.predict(x)

y_pred_linear_test = linear_model.predict(x_test)
y_pred_tree_test = tree_regressor.predict(x_test)
y_pred_knn_test = knn_regressor.predict(x_test)
y_pred_rf_test = random_forest_regressor.predict(x_test)

# y - истинные метки классов
# y_pred - предсказанные метки классов
# y_pred - вероятности классов (для ROC и PR кривых)
models = {'LinearRegression': y_pred_linear, 'DecisionTreeRegressor': y_pred_tree, 'KNeighborsRegressor': y_pred_knn, 'RandomForestRegressor': y_pred_rf}
metrics_train = {}
for model_name, y_pred in models.items():
  # Accuracy
  print(model_name)
  accuracy = accuracy_score(y, y_pred)
  print(f'Accuracy: {accuracy}')

  # F1-score
  f1 = f1_score(y, y_pred)
  print(f'F1-score: {f1}')

  # ROC-AUC
  roc_auc = roc_auc_score(y, y_pred)
  print(f'ROC-AUC: {roc_auc}')

  # PR-AUC
  pr_auc = average_precision_score(y, y_pred)
  print(f'PR-AUC: {pr_auc}')

  # ROC curve
  fpr, tpr, thresholds = roc_curve(y, y_pred)
  #plt.figure()
  #plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc})')
  #plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
  #plt.xlabel('False Positive Rate')
  #plt.ylabel('True Positive Rate')
  #plt.title('ROC Curve')
  #plt.legend(loc='lower right')
  #plt.show()

  # Precision-Recall curve
  #precision, recall, _ = precision_recall_curve(y, y_pred)
  #plt.figure()
  #plt.plot(recall, precision, label=f'PR curve (area = {pr_auc})')
  #plt.xlabel('Recall')
  #plt.ylabel('Precision')
  #plt.title('Precision-Recall Curve')
  #plt.legend(loc='lower left')
  #plt.show()

LinearRegression
Accuracy: 0.9994031038599284
F1-score: 0.998060762766645
ROC-AUC: 0.9985904186143746
PR-AUC: 0.9965236331645191
DecisionTreeRegressor
Accuracy: 0.952049343414246
F1-score: 0.8301620859760396
ROC-AUC: 0.8739058196659545
PR-AUC: 0.7317207600381069
KNeighborsRegressor
Accuracy: 0.9940310385992838
F1-score: 0.9802371541501977
ROC-AUC: 0.9806201550387597
PR-AUC: 0.9672092714782357
RandomForestRegressor
Accuracy: 0.9998010346199762
F1-score: 0.9993535875888817
ROC-AUC: 0.9993540051679586
PR-AUC: 0.9989069757159411


In [ ]:
models = {'LinearRegression': y_pred_linear_test, 'DecisionTreeRegressor': y_pred_tree_test, 'KNeighborsRegressor': y_pred_knn_test, 'RandomForestRegressor': y_pred_rf_test}
metrics_train = {}
for model_name, y_pred in models.items():
  # Accuracy
  print(model_name)
  accuracy = accuracy_score(y_test, y_pred)
  print(f'Accuracy: {accuracy}')

  # F1-score
  f1 = f1_score(y_test, y_pred)
  print(f'F1-score: {f1}')

  # ROC-AUC
  roc_auc = roc_auc_score(y_test, y_pred)
  print(f'ROC-AUC: {roc_auc}')

  # PR-AUC
  pr_auc = average_precision_score(y_test, y_pred)
  print(f'PR-AUC: {pr_auc}')

  # ROC curve
  fpr, tpr, thresholds = roc_curve(y_test, y_pred)
  #plt.figure()
  #plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc})')
  #plt.plot([0, 1], [0, 1], 'k--') # Dashed diagonal
  #plt.xlabel('False Positive Rate')
  #plt.ylabel('True Positive Rate')
  #plt.title('ROC Curve')
  #plt.legend(loc='lower right')
  #plt.show()

  # Precision-Recall curve
  #precision, recall, _ = precision_recall_curve(y_test, y_pred)
  #plt.figure()
  #plt.plot(recall, precision, label=f'PR curve (area = {pr_auc})')
  #plt.xlabel('Recall')
  #plt.ylabel('Precision')
  #plt.title('Precision-Recall Curve')
  #plt.legend(loc='lower left')
  #plt.show()

LinearRegression
Accuracy: 0.9992039800995025
F1-score: 0.9967585089141004
ROC-AUC: 0.9974593596683123
PR-AUC: 0.9941271498390032
DecisionTreeRegressor
Accuracy: 0.964776119402985
F1-score: 0.8545603944124898
ROC-AUC: 0.9117489607222402
PR-AUC: 0.7499539951989731
KNeighborsRegressor
Accuracy: 0.8859701492537313
F1-score: 0.5004359197907584
ROC-AUC: 0.7047443277574732
PR-AUC: 0.31782371202332527
RandomForestRegressor
Accuracy: 0.9793034825870647
F1-score: 0.9097222222222221
ROC-AUC: 0.9228136614080423
PR-AUC: 0.8507246973018259


##**Пункт 6 (Вывод о сравнении метрик)**

При сравнении показателей машинного обучения, наглядно видно, что линейная регрессия (LinearRegression) наиболее точно отображает совпадения в результатах.
Использование других моделей при обучении, способствует улучшению нахождения целевой функции и закономерностей.